In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.26/06


In [2]:
iOption0 = 1 # piminus, electron

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-100M_piminus_single_hagedorn_04-5_v0.root","my-10M_electron_embed_helios_v3.root"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','positron']#output
filename=outfilenames[iOption0]
part_ids=[9,3,2]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"/sphenix/user/mitrankov/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at /sphenix/user/mitrankov/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.8 and myevent.GetEntry(ipart).GetN0()>0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

99865433
Nevent: 14, centrality: 64.0, x: 0.33982399106025696, Ntracks: 1
pt:  0.59 , e/p:  0.947 , n0:  3 , disp:  5.29 , chi2/npe0:  1.95 , prob:  0.86 , id:  positron
0.8460949063301086
Nevent: 27, centrality: 1.0, x: 0.34185999631881714, Ntracks: 1
pt:  0.4 , e/p:  0.876 , n0:  6 , disp:  1.63 , chi2/npe0:  2.47 , prob:  0.21 , id:  positron
0.40519678592681885
Nevent: 28, centrality: 30.0, x: 0.33111199736595154, Ntracks: 1
pt:  1.98 , e/p:  1.059 , n0:  6 , disp:  0.94 , chi2/npe0:  5.79 , prob:  0.827 , id:  positron
1.9908539056777954
Nevent: 29, centrality: 17.0, x: 0.34525901079177856, Ntracks: 1
pt:  0.733 , e/p:  0.897 , n0:  6 , disp:  0.45 , chi2/npe0:  1.26 , prob:  0.012 , id:  positron
0.7465026378631592
Nevent: 38, centrality: 30.0, x: 0.331961989402771, Ntracks: 1
pt:  0.313 , e/p:  1.039 , n0:  5 , disp:  0.93 , chi2/npe0:  1.48 , prob:  0.992 , id:  positron
1.3309226036071777
Nevent: 43, centrality: 22.0, x: 0.3311469852924347, Ntracks: 1
pt:  0.438 , e/p:  1.009 

In [ ]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetPreciseZ(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),myevent.GetEntry(ipart).GetChi2(),myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetPhi0Prime()   ])


data = pd.DataFrame(data_array, columns=('centrality','zvertex','pt', 'e/p', 'n0', 'disp', 'chi2', 'npe0', 'prob', 'phi0'))

99865433


In [ ]:
data.head()

In [ ]:
print(data.size, data.shape, good_event_counter)

In [ ]:
filepath = '/sphenix/user/mitrankov/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [ ]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
parameters = {
    "loss":["log_loss"],
    "learning_rate": [0.01, 0.2],
    "min_samples_split": [0.01, 0.2],
    "max_depth":[3,8],
    "max_features":["log2","sqrt"],
    "criterion": ['squared_error'],
    "n_estimators":[2]
    }
clf3 = GridSearchCV(GradientBoostingClassifier(), parameters, cv=2, n_jobs=4)

X_train = np.array([[0, 0], [1, 1], [0, 1], [1, 0], [0, 1], [1, 0]])
y_train = np.array([0, 1, 0, 0, 0, 1])

# Fit the model
clf3.fit(X_train, y_train)

# Get decision function from clf3
clf_decision = clf3.decision_function(X_train)

# Initialize log odds predictions
log_odds_predictions = np.zeros(X_train.shape[0])

# Get the best estimator
best_estimator = clf3.best_estimator_

# Iterate over trees in the best estimator
for tree in best_estimator.estimators_:
    # Get leaf indices for training data
    leaf_indices = tree[0].apply(X_train)
    # Get values at each leaf (log odds)
    leaf_values = tree[0].tree_.value[:, 0, 0]  # Accessing the log odds correctly
    # Accumulate log odds for each instance
    log_odds_predictions += leaf_values[leaf_indices] * best_estimator.learning_rate

# Final log odds with the learning rate applied
mydecision = log_odds_predictions

# Output the differences
print("My Decision (Log Odds):", mydecision)
print("Decision Function Output:", clf_decision)

# Calculate the differences
difference = np.abs(mydecision - clf_decision)
print("Differences:", difference)
